In [1]:
from datasets import load_dataset
dataset = load_dataset("emotion")

C:\Users\pridd\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No config specified, defaulting to: emotion/split
Found cached dataset emotion (C:/Users/pridd/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd)
100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 744.95it/s]


In [2]:
df=dataset
df

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [3]:
train=df['train']
val=df['validation']
test=df['test']

In [4]:
def get_tweet(data):
    tweet=[x['text'] for x in data]
    label=[x['label'] for x in data]
    return tweet,label

In [5]:
tweet,label=get_tweet(train)

In [6]:
tweet[4],label[4]

('i am feeling grouchy', 3)

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [8]:
tokenizer=Tokenizer(num_words=1000,oov_token='<UNK>')
tokenizer.fit_on_texts(tweet)

In [9]:
tokenizer.texts_to_sequences([tweet[0]])

[[2, 139, 3, 679]]

In [10]:
maxlen=50

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
def get_sequences(tokenizer,tweet):
    sequences=tokenizer.texts_to_sequences(tweet)
    padded=pad_sequences(sequences,truncating='post',padding='post',maxlen=maxlen)
    return padded

In [12]:
padded_train_seq=get_sequences(tokenizer,tweet)

In [13]:
padded_train_seq[0]

array([  2, 139,   3, 679,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [14]:
classes=set(label)
classes

{0, 1, 2, 3, 4, 5}

In [15]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Dropout,LSTM
from keras.models import Sequential

In [16]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Embedding(1000,16,input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
    tf.keras.layers.Dense(6,activation='softmax')
])


model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 16)            16000     
                                                                 
 bidirectional (Bidirectiona  (None, 50, 40)           5920      
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 40)               9760      
 nal)                                                            
                                                                 
 dense (Dense)               (None, 6)                 246       
                                                                 
Total params: 31,926
Trainable params: 31,926
Non-trainable params: 0
_________________________________________________________________


In [18]:
val_tweet,val_label=get_tweet(val)

In [19]:
val_seq=get_sequences(tokenizer,val_tweet)

In [20]:
import numpy as np
padded_train_seq = np.array(padded_train_seq)
label = np.array(label)
val_seq = np.array(val_seq)
val_label = np.array(val_label)

In [21]:
h=model.fit(padded_train_seq,label,validation_data=(val_seq,val_label),epochs=100)

Epoch 1/100
500/500 [==============================] - 20s 29ms/step - loss: 1.3051 - accuracy: 0.4814 - val_loss: 0.8948 - val_accuracy: 0.6445
Epoch 2/100
500/500 [==============================] - 14s 28ms/step - loss: 0.6838 - accuracy: 0.7602 - val_loss: 0.5903 - val_accuracy: 0.8010
Epoch 3/100
500/500 [==============================] - 16s 32ms/step - loss: 0.4947 - accuracy: 0.8313 - val_loss: 0.5238 - val_accuracy: 0.8090
Epoch 4/100
500/500 [==============================] - 15s 29ms/step - loss: 0.4157 - accuracy: 0.8531 - val_loss: 0.4547 - val_accuracy: 0.8395
Epoch 5/100
500/500 [==============================] - 15s 29ms/step - loss: 0.3729 - accuracy: 0.8662 - val_loss: 0.5058 - val_accuracy: 0.8310
Epoch 6/100
500/500 [==============================] - 15s 29ms/step - loss: 0.3500 - accuracy: 0.8729 - val_loss: 0.4173 - val_accuracy: 0.8540
Epoch 7/100
500/500 [==============================] - 14s 28ms/step - loss: 0.3188 - accuracy: 0.8829 - val_loss: 0.4140 - val_ac

KeyboardInterrupt: 